In [3]:
# Load imports
%run 101_imports.ipynb

<Figure size 432x288 with 0 Axes>

In [4]:
# Load imports
%run 102_log_to_file.ipynb

In [5]:
# Replace missing values 
%run 103_missing_values.ipynb

In [6]:
# Display all columns
%run 104_display_all_columns.ipynb

In [8]:
# Read file
%run 105_read_file.ipynb

In [10]:
# Remove table prefix after reading from database
%run 106_replace_prefix.ipynb

In [12]:
# Group and sort
%run 107_groupby_sort.ipynb

In [1]:
# Remove duplicates
%run 108_duplicate_remove.ipynb

In [3]:
# Separate numerical and categorical columns
%run 111_separate_numeric_categoric.ipynb

In [4]:
# Create dummy for categorical columns
%run 112_create_dummy.ipynb

In [5]:
# Display facegrid histogram
%run 113_facegrid.ipynb

In [7]:
# Aggregate data using aggregate functions like max or latest. 
# For example if column is age ,max age will be taken.
%run 114_aggregate.ipynb

In [8]:
# Covariance 
%run 115_covrtn.ipynb

In [10]:
# Drop correlated columns
%run 121_drop_correlated_columns.ipynb 

In [12]:
# Split data into train and test data
%run 122_split.ipynb

In [13]:
# Train the model
%run 123_train_model.ipynb

In [14]:
# Save model
%run 124_save_model.ipynb

In [15]:
# Write to file
%run 125_write_file.ipynb

In [9]:
# Continue from here

In [8]:
def trend_mean_std_CoV(df):
    df = df.sort_values(by=['p_inco','file_date'],ascending = True)
    fet_cols = ['trend','mean','std','CoV']
    base_features_ls = pd.DataFrame()
    base_features_ls['p_icno'] = pd.unique(df.p_icno)
    base_features_ls['row_id'] = (df.row_id)
    return(base_features_ls)
# Continue from here.

In [23]:
############### Only SMOTE  ############################################################
# Over-sampling: SMOTE
def smote(X, y):
    from imblearn.over_sampling import SMOTE
    
    smote = SMOTE(ratio='minority')
    X_sm, y_sm = smote.fit_sample(X, y)
    
    plot_2d_space(X_sm, y_sm, 'SMOTE over-sampling')
    
    return X_sm, y_sm

################  SMOTE and Tomek ##############################################################
# Over-sampling followed by under-sampling
# Now, we will do a combination of over-sampling and under-sampling, using the SMOTE and Tomek links techniques:
def smote_and_tomek(X, y):
    from imblearn.combine import SMOTETomek
    
    smt = SMOTETomek(ratio='auto')
    X_smt, y_smt = smt.fit_sample(X, y)
    
    plot_2d_space(X_smt, y_smt, 'SMOTE + Tomek links')
    
    return X_smt, y_smt

In [24]:
# Split validation into X and y
def split_target(target,df):
    y = df[target]
    X = df.drop(target,axis = 1)        
    return X,y

# Feature importance
def feature_importance(X,model):   
    importances = model.feature_importances_
    std = np.std([tree.feature_importances_ for tree in model.estimators_],axis=0)
    indices = np.argsort(importances)[::-1]
    
    print(" \n Data Set shape is: \n",X.shape[0],X.shape[1])
    for f in range(X.shape[1]):
        print("%d. feature %d %s (%f)" % (f + 1, indices[f],X.columns.values[indices[f]],importances[indices[f]]))
    
    feat_importances = pd.Series(model.feature_importances_, index=X.columns)
    feat_importances.nlargest(X.shape[1]).plot(kind='barh')
    feat_importances.sort_values(ascending =False) #.head(5)

In [14]:
def predict_results(model,X_test,y_test):
    pred = model.predict(X_test)
    if(algo != 'SV'):
        pred_proba = model.predict_proba(X_test)
        pred_proba = [p[1] for p in pred_proba]
        
    conf_matrix =confusion_matrix(y_test,pred)
    clas_report = classification_report(y_test,pred)
    acc_score =  accuracy_score(y_test,pred)
    if(algo != 'SV'):
        return pred,pred_proba,conf_matrix,clas_report,acc_score
    return pred,conf_matrix,clas_report,acc_score

In [15]:
def roc_curve_plot(y_test,pred):
    from sklearn.metrics import auc,log_loss,roc_curve,confusion_matrix
    import matplotlib.pyplot as plt
    
    fpr,tpr,threshold = roc_curve(y_test,pred)
    roc_auc = auc(fpr,tpr)
    plt.title("ROC")
    plt.plot(fpr,tpr,'b',label='AUC=%0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0,1],[0,1],'r--')
    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.show()
    return plt          
              

In [16]:
def grid_search(X_test,y_train):
    from sklearn.model_selection import GridSearchCV
    from sklearn.ensemble import RandomForestClassifier
    params = {'n_estimators':[10,15,20,25],
              'max_depth':[10,12,14],
              'min_sample_lead':[100,120,140],
              'max_features':[0.1,0.2,0.3]}
    
    grid = GridSearchCV(estimators = RandomForestClassifier(random_state =21),
                       param_grid = params,
                       cv = 5,
                       scoring = 'f1',
                       n_jobs = 1,
                       verbose = 3)
    grid.fit(X_test,y_train)
    return grid.best_parameters_

In [17]:
# # working one
# def decile_analysis(qq,algo):
#     print(' before call df_decile: \n',qq.head(),qq.shape)
    
#     base_response_rate = np.round(100*qq[qq.y_true==1].shape[0]/qq.shape[0],decimals=2)
#     print('base_response_rate : \n',base_response_rate)
    
#     qq.sort_values(by='y_prob',inplace = True,ascending = False)
#     qq.reset_index(inplace = True)
#     qq['decile'] = np.nan
#     d =int(np.ceil(qq.shape[0]/10))
#     start = 0
#     end = d
#     for i in range(10):
#         qq.loc[start:end,['decile']] = i+1
#         start = start +d
#         end = end +d
#         qq = pd.crosstab(qq['decile'],qq['y_true'])
#         qq.columns = ['zero','one']
        
#         print('qq: \n',qq)        
#         qq['min_prob'] = qq.groupby(by=['decile']).min()['y_prob']
#         qq['max_prob'] = qq.groupby(by=['decile']).max()['y_prob']
#         qq['count'] = qq.groupby(by=['decile']).count()['y_prob']
#         qq['gain'] = np.round(100*qq['one']/qq['one'].sum(),decimals=2)
#         qq['cum_gain'] = np.cumsum(qq['gain'])
#         qq['penetration'] = (qq['one']/qq['count'])*100
#         print('final qq: \n',qq) 
#         return qq

In [18]:
# def decile_analysis(qq,algo):
#     print('algo: \n',algo)
#     base_response_rate = np.round(100*qq[qq.y_true==1].shape[0]/qq.shape[0],decimals=2)
#     print('base_response_rate : \n',base_response_rate)
# #     if(algo != 'SV'):
# #         qq.sort_values(by='y_prob',inplace = True,ascending = False)
#     qq.sort_values(by='y_prob',inplace = True,ascending = False)
#     qq.reset_index(inplace = True)
#     qq['decile'] = np.nan
#     d =int(np.ceil(qq.shape[0]/10))
#     start = 0
#     end = d
#     for i in range(10):
#         qq.loc[start:end,['decile']] = i+1
#         start = start +d
#         end = end +d
#         pd.crosstab(qq['decile'],qq['y_true'])
#         print('qq: \n',qq)
#         qq.columns = ['zero','one']
# #         if(algo != 'SV'):
# #             qq['min_prob'] = qq.groupby(by==['decile']).min()['y_prob']
# #             qq['max_prob'] = qq.groupby(by==['decile']).max()['y_prob']
# #             qq['count'] = qq.groupby(by==['decile']).count()['y_prob']
#         qq['min_prob'] = qq.groupby(by==['decile']).min()['y_prob']
#         qq['max_prob'] = qq.groupby(by==['decile']).max()['y_prob']
#         qq['count'] = qq.groupby(by==['decile']).count()['y_prob']
            
#         qq['gain'] = np.round(100*qq['one']/qq['one'].sum(),decimals=2)
#         qq['cum_gain'] = np.cumsum(qq['gain'])
#         qq['penetration'] = (qq['one']/qq['count'])*100
#         return qq